In [1]:
import pandas as pd
import numpy as np
import random 
import matplotlib.pyplot as plt

from sklearn import svm

dataset = ["titanic_train","iris"]

In [2]:
def LeerDatos(filename : str, separa : str, header = True):
    if (header):
        data = pd.read_csv(filename + ".csv", sep =separa, header = 0)
    else:
        data = pd.read_csv(filename+ ".csv", sep = separa, header = None)
    return data.to_numpy()

In [3]:
def Normalizar_Datos(data : np.array):
    #normal = np.empty_like(data)
    for i in range (0,np.size(data[0])):
        media = np.mean(data[:,i])
        desvi =np.std(data[:,i])
        data[:,i] = (data[:,i] - media)/desvi
    return data

In [4]:
def Crear_k_folds(data : np.array , k:int, clases: []):
    folds = []
    tot_clase = []
    prop_clase = [] #Acumulado de indices
    pre_fold = []

    m = np.size(data[:,-1]) #numero de datos
    #n = np.size(data[0])
    for i in clases:
        tot_clase.append(np.count_nonzero( data[:,-1] == i))

    prop_clase.append(tot_clase[0])
    for i in range (1, len(tot_clase)):
        prop_clase.append( prop_clase[i-1] + tot_clase[i])

    pos_ini = 0
    for i in range(0, len(clases)):
        pre_fold.append(np.array_split(data[pos_ini:prop_clase[i]], k))
        pos_ini = prop_clase[i]
    
    for i in range (0,k):
        temp = np.empty( (0,np.size(data[0])) )
        for j in range(0,len(clases)):
            temp = np.vstack( (temp,pre_fold[j][i]))
        folds.append(temp)
            
    return folds

In [5]:
def TransformacionClases(y:np.array, clases:[]):
    num_clases = len(clases)
    #vec_clases = np.empty((0), dtype = int)
    vec_clases = []
    for i in y:
        idx = clases.index(i)
        vec_clases.append(idx)
    return vec_clases

In [6]:
def Calcular_Accuracy(X:np.array, y:np.array, model):
    y_calc = model.predict(X)
    
    aciertos = 0
    for i in range(0, len(y)):
        if (y[i] == y_calc[i]):
            aciertos += 1
    return aciertos/len(y)
    
def PromedioAccuracy(test:np.array, model, k, clases):
    accu = np.zeros(k)
    for i in range(0,k):
        X_test = test[i][:,:-1]
        y_test = TransformacionClases(test[i][:,-1], clases)
        accu[i] = Calcular_Accuracy(X_test, y_test, model)
    return accu.mean()

In [7]:
def CalculoParametrosSVM(folds:[], k:int, C:float, kernel:str , clases:[]):
    arr_costo = []
    arr_theta = []
    arr_test = []
    for test_i in range(0, k):
        test = folds[test_i] 
        train = np.zeros( (0,np.size(folds[0][0])) )
        for train_i in range (0, k):         
            if (train_i == test_i):
                continue
            else:
                train = np.vstack( (train,folds[train_i]) )
            
        #costo = []
        X_train = train[:,:-1]
        #X_train = X_train.astype('float64')
        #X_train = Normalizar_Datos(X_train)

        y_train = TransformacionClases( train[:,-1], clases)
        
        model = svm.SVC(kernel = kernel, C=C, degree=2)
        model.fit(X_train, y_train)
        arr_test.append(test)
      
    return model, arr_test

In [8]:
#EXPERIMENTO 3
#1.1 TITANIC 

In [21]:
#Leer solo los datos necesarios
titanic = LeerDatos(dataset[0],',')
titanic_train = np.c_[titanic[:,0:2],titanic[:,4:6],titanic[:,-3:-2],titanic[:,-1:]]

#quitamos los datos nulos
titanic_train = pd.DataFrame(titanic_train).dropna()
titanic_train = titanic_train.sort_values(titanic_train.columns[1])

#dividimos para normalizar 
ids  = titanic_train.to_numpy()[:,0:1]
survived  = titanic_train.to_numpy()[:,1:2]
sex  = titanic_train.to_numpy()[:,2:3]
age  = titanic_train.to_numpy()[:,3:4]
fare = titanic_train.to_numpy()[:,4:5]
embarked = titanic_train.to_numpy()[:,5:]

#reemplazamos female/male por 1/0
sex = pd.DataFrame(sex).replace({"male": 0, "female": 1})
#reemplazamos Q/S/C por 1/2/3
embarked = pd.DataFrame(embarked).replace({"Q": 1, "S": 2, "C": 3})

#juntamos la data
titanic_train_ = np.array(np.c_[ids,sex,age,fare,embarked,survived]) #id,survived,sex,age,fare,embarked 

#clase y num_clases 
clases = [0,1]
num_clases = len(clases);

#k folds 
k = 3

#creamos los folds
fold_titanic = Crear_k_folds(titanic_train_,k,clases)

#Kernels
kernels = ['linear', 'poly', 'rbf']
coef = np.arange(2.0,5.1,0.5)
Matriz_accurracy_prom = np.empty( (len(kernels),len(coef) ))

print("Dataset TITANIC")
for ker in range(0,len(kernels)):
    for C in range(0, len(coef)):
        model, test = CalculoParametrosSVM(fold_titanic, k, coef[C], kernels[ker], clases)
        Matriz_accurracy_prom[ker,C] = PromedioAccuracy(test, model, k, clases)
print(pd.DataFrame(Matriz_accurracy_prom, index = kernels, columns = coef))

Dataset TITANIC
             2.0       2.5       3.0       3.5       4.0       4.5       5.0
linear  0.780898  0.776708  0.764091  0.755652  0.748632  0.743006  0.738787
poly    0.624999  0.634838  0.632037  0.633443  0.639051  0.641864  0.644677
rbf     0.540439  0.541845  0.541845  0.544664  0.544658  0.546065  0.547471


In [9]:
#1.2 ESPECIES DE FLORES(IRIS Setosa -Iris versicolor- Iris virginica) 
#Dataset contiene 5 columnas 1-Longitud del sépalo en centímetros. 2-Ancho del sépalo en centímetros.
#3-Longitud del pétalo en centímetros. 4-Ancho del pétalo en centímetros.
#5-Clase.
# url https://unipython.com/clasificacion-multiclase-de-especies-de-flores/

#iris = LeerDatos(dataset[0], separa = ',')
#clases = ["Setosa", "Versicolor", "Virginica"]
#iris_folds = Crear_k_folds(iris, 3, clases)
#theta, arr_costo, arr_test = CalculoParametros(iris_folds, 3, 500, 0.1, 3, 3, 20, clases)
especie_flores = LeerDatos(dataset[1], separa = ',')
#quitamos los datos nulos ordenamos por clase 
especie_flores = pd.DataFrame(especie_flores).dropna()
especie_flores = especie_flores.sort_values(especie_flores.columns[-1])

aux = especie_flores.to_numpy()[:,0:4]
clase  = especie_flores.to_numpy()[:,4:]

clase = pd.DataFrame(clase).replace({"Iris-setosa": 1, "Iris-versicolor": 2, "Iris-virginica": 3})


#juntamos la data
especie_flores = np.array(np.c_[aux,clase]) #id,survived,sex,age,fare,embarked 

#clase y num_clases 
clases = [1,2,3]
num_clases = len(clases);

#k folds 
k = 3

#creamos los folds
fold_iris = Crear_k_folds(especie_flores,k,clases)

#Kernels
kernels = ['linear', 'poly', 'rbf']
#C
coef = np.arange(2.0,5.1,0.5)

Matriz_accurracy_prom = np.empty( (len(kernels),len(coef) ))

print("Dataset - Especie de flores")
for ker in range(0,len(kernels)):
    for C in range(0, len(coef)):
        model, test = CalculoParametrosSVM(fold_iris, k, coef[C], kernels[ker], clases)
        Matriz_accurracy_prom[ker,C] = PromedioAccuracy(test, model, k, clases)
print(pd.DataFrame(Matriz_accurracy_prom, index = kernels, columns = coef))


Dataset - Especie de flores
             2.0       2.5       3.0       3.5       4.0       4.5       5.0
linear  0.979575  0.979575  0.973039  0.973039  0.973039  0.973039  0.973039
poly    0.986520  0.986520  0.986520  0.979575  0.979575  0.979575  0.973039
rbf     0.973725  0.986928  0.993464  0.993464  0.993464  0.993464  0.993464
